<a href="https://colab.research.google.com/github/Molten-Ice/Deep-Learning/blob/dev/GPT_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook I will be coding a GPT from scratch. 

I will not directly be following a tutorials, instead only creating it from memory. 

It's core component in Transformers, more precisely attention.

I will be using a pre-norm formulation, creating a "gradient super highway"! Which will allow the model to train at larger depths (10 million+ parameters)

In [ ]:
### Prompts
# residual connections are super important
# linearly project multi-head attention output, then dropout

#feed forward linear(n, 4n), GeLU, linear(4n, n), dropout

#pre norm formulation, creates gradient super highway!
#layer norm before it goes into self-attention and feedforward

#add layer norms after block before final linear layer

#scaling up module
#dropout after softmax

In [4]:
try:
  import einops
except:
  print("einops not installed, installing...")
  !pip install einops
  import einops

einops not installed, installing...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 1.7 MB/s eta 0:00:00


In [6]:
import torch
import torch.nn as nn

In [7]:
# hyperparameters
batch_size = 64 # num independent sequences processed in parallel 
block_size = 256 # what is the maximum context lengths?

eval_interval = 500 # how often to print out loss & accuracy
eval_iters = 200 # how many sequences to check during evaluation

max_iters = 5000 # training iterations
learning_rate = 3e-4
dropout = 0.2

n  = 8
n_embedding = 512 # each head has dim 64 (=512/8)
n_layer = 6

device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [11]:
# Importing data
file = "foundation.txt"
with open(file, 'r') as f:
  text = f.read()

print(f"Length of {file}: {len(file)}")

Length of foundation.txt: 14


In [10]:
text

'FOUNDATION \nISAAC ASIMOV \n\nPART I \n\nTHE PSYCHOHISTORIANS \n\ni. \n\nHARI SELDON-... bom In the 1 1,988th year of the Galactic Era; died 12,069. The dates are \nmore commonly given In terms of the current Foundational Era as - 79 to the year 1 F.E. Born \nto middle-class parents on Flelicon, Arcturus sector (where his father, In a legend of doubtful \nauthenticity, was a tobacco grower in the hydroponic plants of the planet), he early showed \namazing ability in mathematics. Anecdotes concerning his ability are innumerable, and some \nare contradictory. At the age of two, he is said to have ... \n\n... Undoubtedly his greatest contributions were in the field of psychohistory. Seldon found the \nfield little more than a set of vague axioms; he left it a profound statistical science.... \n\n... The best existing authority we have for the details of his life is the biography written by Gaal \nDornick who. as a young man, met Seldon two years before the great mathematician\'s death. \